# Transformers

Baseline for now

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")
    
# Load the dataset
df = pd.read_csv('../dataset.csv')

# Load a pre-trained SentenceTransformer model
model = SentenceTransformer('all-mpnet-base-v2', device=device)

# Generate embeddings for the combined features
print("Generating embeddings...")

# Generate separate embeddings
embeddings = model.encode(df['content'].tolist(), show_progress_bar=True)

# Calculate cosine similarity
cosine_sim = cosine_similarity(embeddings, embeddings)
# Create a reverse map of indices and movie titles
indices = pd.Series(df.index, index=df['tconst']).drop_duplicates()

/Users/tomasmalamud/miniconda3/envs/recommender-test/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Using MPS


/Users/tomasmalamud/miniconda3/envs/recommender-test/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating embeddings...


Batches: 100%|██████████| 237/237 [10:28<00:00,  2.65s/it]


In [30]:
# Save embeddings to a file
import numpy as np

print("Saving embeddings...")
np.save('embeddings.npy', embeddings)

print("Embeddings saved successfully.")


Saving embeddings...
Embeddings saved successfully.


In [56]:
def get_recommendations(tconst, df=df, embeddings=embeddings_better, top_n=30, content_weight=0.9, popularity_weight=0.1):
    idx = indices[tconst]
    movie_embedding = embeddings[idx].reshape(1, -1)
    
    # Compute cosine similarity on-the-fly
    cosine_sim = cosine_similarity(movie_embedding, embeddings)[0]
    
    sim_scores = []
    for i, content_sim in enumerate(cosine_sim):
        combined_sim = (
            content_weight * content_sim +
            popularity_weight * df['popularity_score'].iloc[i]
        )
        sim_scores.append((i, combined_sim))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Get top N+1 (excluding the movie itself)
    
    movie_indices = [i[0] for i in sim_scores]
    
    recommendations = df.iloc[movie_indices]
    recommendations['similarity'] = [i[1] for i in sim_scores]

    return recommendations


In [57]:
# Test the function
movie_tconst = 'tt1596363'  # Fight Club
recommendations = get_recommendations(movie_tconst, top_n=30)
print(recommendations['title'])


2305         Wall Street: Money Never Sleeps
16                   The Wolf of Wall Street
2974                              Inside Job
1693                             Margin Call
1498                             Wall Street
4325                              Dumb Money
2805                     The Hudsucker Proxy
2313                           Money Monster
3946                               Arbitrage
1482                          Trading Places
57                       Catch Me If You Can
623                           The Accountant
4858                       Small Time Crooks
836                               RocknRolla
7498    Enron: The Smartest Guys in the Room
7184                  Assault on Wall Street
2168                                Paycheck
154                          American Psycho
1639                             Tower Heist
773                                The Sting
3352                          The Informant!
1188                              Goldfinger
2438      

/var/folders/5v/nhcrhk997_lfsv2px600ylsw0000gn/T/ipykernel_2333/696735000.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['similarity'] = [i[1] for i in sim_scores]
